In [47]:
import pandas as pd
import numpy as np
from numpy import nan


In [48]:
df = pd.read_csv("jobs18.csv")

In [49]:
df.head()

,Unnamed: 0,title,company,salary,location,company_rating,description
0,0,Junior Data Scientist Apprentice,IBM,NaN,"New York, NY 10001 (Chelsea area)+1 location",3.9,Introduction\nAs a Junior Data Scientist at IB...
1,1,Data Scientist,Adidas,NaN,"Portland, OR+1 location",4.0,Purpose & Overall Relevance for the Organizati...
2,2,Junior Data Scientist (NYC or Remote),ViacomCBS,NaN,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...
3,3,"Associate Data Scientist, User Fraud",Spotify,NaN,"Hybrid remote in New York, NY+14 locations",4.3,Content\nData\nWe’re looking for an Associate ...
4,4,"Data Scientist, Game Analytics & Strategy",the NBA,NaN,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and..."


In [50]:
df.shape

(1475, 7)

We will start to work from salary column. Goal here is to have 
1) mean value for the salary if they have a salary estimation
2) or leave it as NaN

First Let's figure out
- How many NaN value we have
- How many value that we can use
- structure of non NaN values
- extract only numbers

In [51]:
print('we have', df['salary'].isna().sum(), 'Nan value for the salary column')
print('that indicates that we have only 56 values to use')

we have 1330 Nan value for the salary column
that indicates that we have only 56 values to use


In [52]:
salary = df['salary']
salary_notnull = salary[salary.notnull()]
print(salary_notnull)


14      $89,457 - $115,262 a year
18      $92,700 - $160,000 a year
26      $70,400 - $117,400 a year
29                $110,500 a year
32      $94,727 - $122,885 a year
                  ...            
1425    $83,200 - $190,100 a year
1430              $125,000 a year
1434    $78,300 - $117,500 a year
1460    $95,800 - $159,600 a year
1464    $78,300 - $117,500 a year
Name: salary, Length: 145, dtype: object


Now let's extract numbers only from the salary columns

In [53]:
def str_contains_n(str):
    num_str = "0123456789"
    word_list = []
    words = str.split(" ")
    for word in words:
        word_has_n = False
        for char in word:
            if char in num_str:
                word_has_n = True
        if word_has_n:
            word_list.append(word)
    result = '-'.join(word_list)
    return result

In [54]:
df['k_unit'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'k' in x.lower() else 0)
df['yearly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a year' in x.lower() else 0)
df['monthly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'a month' in x.lower() else 0)
df['hourly'] = df['salary'].apply(lambda x: 1 if isinstance(x, str) and 'an hour' in x.lower() else 0)

salary = salary.apply(lambda x: str_contains_n(x) if isinstance(x, str) else None)
salary = salary.apply(lambda x: x.lower().replace("$","").replace("k","").replace(",","") if x != None else None)
salary = salary.apply(lambda x: x if isinstance(x,str) else 0)

df['salary_edit'] = salary

df['min_salary'] = salary.apply(lambda x: float(x.split('-')[0]) if isinstance(x,str) else 0)
df['max_salary'] = salary.apply(lambda x: float(x.split('-')[1]) if isinstance(x,str) and len(x.split('-')) >1 else(float(x.split('-')[0]) if isinstance(x,str) else 0))
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

# take care of k_unit
df.loc[df['k_unit'] == 1,['min_salary']] = df[df['k_unit'] == 1].min_salary * 1000
df.loc[df['k_unit'] == 1,['max_salary']] = df[df['k_unit'] == 1].max_salary * 1000
df.loc[df['k_unit'] == 1,['avg_salary']] = df[df['k_unit'] == 1].avg_salary * 1000
df.loc[df['k_unit'] == 1]

# take care of monthly, for the monthly we have to multiply 12
df.loc[df['monthly'] == 1,['min_salary']] = df[df['monthly'] == 1].min_salary * 12
df.loc[df['monthly'] == 1,['max_salary']] = df[df['monthly'] == 1].max_salary * 12
df.loc[df['monthly'] == 1,['avg_salary']] = df[df['monthly'] == 1].avg_salary * 12
df.loc[df['monthly'] == 1]

df.loc[df['hourly'] == 1,['min_salary']] = df[df['hourly'] == 1].min_salary * 40 * 52
df.loc[df['hourly'] == 1,['max_salary']] = df[df['hourly'] == 1].max_salary * 40 * 52
df.loc[df['hourly'] == 1,['avg_salary']] = df[df['hourly'] == 1].avg_salary * 40 * 52
df.loc[df['hourly'] == 1]

# df.at[df['k_unit'] == 1, 'Product_Name'] = df[df['k_unit'] == 1].min_salary * 1000
# Now that we have get the yearly average salary correctly we can drop all unneccessary columns

df.drop(['k_unit','yearly','monthly','hourly','salary_edit','min_salary','max_salary','salary'], axis=1, inplace = True)
# pd.set_option("display.max_rows", None, "display.max_columns", None)




Now we need to clean up the location columns
there are few information we can extract
1. remote or not
2. state name

1) we can compare whether remote job gives more salary or the onsite job
2) which state gives are more salary
3) even with the remote job, does the state gives out high salary still gives are high salary

the above questions we can answer later on

In [55]:

df['remote'] = df['location'].apply(lambda x: 1 if isinstance(x,str) and 'remote' in x.lower() else 0)
df.groupby('remote').count()
# we can see that 158/450 jobs are remote

,Unnamed: 0,title,company,location,company_rating,description,avg_salary
remote,,,,,,,
0,1071,1071,1071,1071,861,369,1071
1,404,404,404,404,294,130,404


In [56]:
# Let's extract state name
# from https://gist.github.com/JeffPaine/3083347 I copied the state acronymn list
states_ac = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states_dict = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

states = []
for state in states_dict:
    states.append(states_dict[state])

states_all = states_ac + states



In [57]:
import re
newData = "GeeksforGeeks, is_an-awesome ! app + too"
  
# To split "+" use backslash
def check_contains_ele(location, state_list):
    location = location.replace('in',"")
    location_list = re.split(' |\+',location)
    print(location_list)
    for location in location_list:
         for state in state_list:
             if location.lower() == state.lower():
                 return state
    return 0
    
df['state'] = df['location'].apply(lambda x: check_contains_ele(x, states_all) if isinstance(x,str)  else 0)
df[df['state'] == 0].head()

['New', 'York,', 'NY', '10001', '(Chelsea', 'area)', '1', 'location']
['Portland,', 'OR', '1', 'location']
['Remote', '', 'New', 'York,', 'NY', '1', 'location']
['Hybrid', 'remote', '', 'New', 'York,', 'NY', '14', 'locations']
['New', 'York,', 'NY', '10022', '(Midtown', 'area)', '1', 'location']
['New', 'York,', 'NY', '10017', '(Midtown', 'area)', '35', 'locations']
['Redmond,', 'WA', '98052', '(Overlake', 'area)', '31', 'locations']
['Beaverton,', 'OR', '2', 'locations']
['Atlanta,', 'GA', '9', 'locations']
['Alpharetta,', 'GA', '6', 'locations']
['Sacramento,', 'CA', '1', 'location']
['New', 'York,', 'NY']
['Washgton,', 'DC', '20001', '(Swampoodle', 'area)', '1', 'location']
['Remote', '', 'New', 'York,', 'NY', '3', 'locations']
['Washgton,', 'DC', '20005', '(Downtown', 'area)', '1', 'location']
['San', 'Jose,', 'CA', '7', 'locations']
['New', 'York,', 'NY']
['Alpharetta,', 'GA', '6', 'locations']
['', '126', 'locationsRemote']
['Charlotte,', 'NC', '28269', '6', 'locations']
['Santa'

,Unnamed: 0,title,company,location,company_rating,description,avg_salary,remote,state
18,18,Data Scientist I,Amazon.com Services LLC,+126 locationsRemote,3.5,"M.S. or equivalent in Statistics, Mathematics,...",126350.0,1,0
30,30,(Remote) Data Scientist,Outten & Golden,Remote,4.0,Outten & Golden is accepting applications for ...,0.0,1,0
40,40,Data Scientist - REMOTE,Appen Butler Hill Inc.,Remote,NaN,Overview:\nDATA SCIENTIST\nPosition Summary\nA...,0.0,1,0
42,42,Green Energy Data Scientist,Piper Companies,+3 locationsRemote,4.5,Piper Companies is seeking a Green Energy Data...,125000.0,1,0
51,51,Data Scientist (Remote),XSELL Technologies,Remote,3.5,About XSELL\nReady to write the best chapter o...,0.0,1,0


In [58]:
# python
# aws
# gcp
# excel
# spark

df['python'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'python' in x.lower() else 0)
df['aws'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'aws' in x.lower() else 0)
df['gcp'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'gcp' in x.lower() else 0)
df['excel'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'excel' in x.lower() else 0)
df['spark'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'spark' in x.lower() else 0)
df['sql'] = df['sql'] = df['description'].apply(lambda x: 1 if isinstance(x,str) and 'sql' in x.lower() else 0)

df.columns
df.drop('Unnamed: 0',axis = 1, inplace = False)
df.to_csv('job_data_cleaned.csv')
df

,Unnamed: 0,title,company,location,company_rating,description,avg_salary,remote,state,python,aws,gcp,excel,spark,sql
0,0,Junior Data Scientist Apprentice,IBM,"New York, NY 10001 (Chelsea area)+1 location",3.9,Introduction\nAs a Junior Data Scientist at IB...,0.0,0,NY,1,0,0,0,1,0
1,1,Data Scientist,Adidas,"Portland, OR+1 location",4.0,Purpose & Overall Relevance for the Organizati...,0.0,0,OR,1,1,0,1,0,1
2,2,Junior Data Scientist (NYC or Remote),ViacomCBS,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...,0.0,1,NY,1,0,0,0,0,1
3,3,"Associate Data Scientist, User Fraud",Spotify,"Hybrid remote in New York, NY+14 locations",4.3,Content\nData\nWe’re looking for an Associate ...,0.0,1,NY,1,0,0,0,0,1
4,4,"Data Scientist, Game Analytics & Strategy",the NBA,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and...",0.0,0,NY,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,1470,Data Scientist,Hyperspace Ventures,"Durham, NC",NaN,NaN,0.0,0,NC,0,0,0,0,0,0
1471,1471,Principal Data Scientist,Schneider Electric,"Boston, MA",3.9,NaN,0.0,0,MA,0,0,0,0,0,0
1472,1472,Data Scientist,"TTI, INC.","Fort Worth, TX 76106 (Northside area)",3.6,NaN,0.0,0,TX,0,0,0,0,0,0
1473,1473,Principal Data Scientist,Silicon Valley Bank,"Tempe, AZ+2 locations",3.8,NaN,0.0,0,AZ,0,0,0,0,0,0


In [59]:
import matplotlib.pyplot as plt
import seaborn as sns


In [60]:
df.head()

,Unnamed: 0,title,company,location,company_rating,description,avg_salary,remote,state,python,aws,gcp,excel,spark,sql
0,0,Junior Data Scientist Apprentice,IBM,"New York, NY 10001 (Chelsea area)+1 location",3.9,Introduction\nAs a Junior Data Scientist at IB...,0.0,0,NY,1,0,0,0,1,0
1,1,Data Scientist,Adidas,"Portland, OR+1 location",4.0,Purpose & Overall Relevance for the Organizati...,0.0,0,OR,1,1,0,1,0,1
2,2,Junior Data Scientist (NYC or Remote),ViacomCBS,"Remote in New York, NY+1 location",3.9,ViacomCBS (NASDAQ:\nVIAC; VIACA) is a leading ...,0.0,1,NY,1,0,0,0,0,1
3,3,"Associate Data Scientist, User Fraud",Spotify,"Hybrid remote in New York, NY+14 locations",4.3,Content\nData\nWe’re looking for an Associate ...,0.0,1,NY,1,0,0,0,0,1
4,4,"Data Scientist, Game Analytics & Strategy",the NBA,"New York, NY 10022 (Midtown area)+1 location",4.2,"At the NBA, we’re passionate about growing and...",0.0,0,NY,1,0,0,1,0,1


In [61]:
df.columns

Index(['Unnamed: 0', 'title', 'company', 'location', 'company_rating',
       'description', 'avg_salary', 'remote', 'state', 'python', 'aws', 'gcp',
       'excel', 'spark', 'sql'],
      dtype='object')

In [74]:
df["title"].value_counts()
# job title parsing
def jobtitle_cleaning(title):
    title = title.lower()
    if 'data scientist' in title:
        return "data scientist"
    elif 'data engineer' in title:
        return 'data engineer'
    elif 'machine learning' in title:
        return 'mle'
    elif 'analyst' in title:
        return 'analyst'
    elif 'manager' in title:
        return 'manager'
    elif 'director' in title:
        return 'director'
    else:
        return 'na'
    
def seniority_cleaning(title):
    seniors = ['sr','senior','sr.','lead','principal']
    juniors = ['jr', 'jr.', 'junior']
    title = title.lower()
    for senior in seniors:
        if senior in title:
            return 'senior'
    for junior in juniors:
        if junior in title:
            return 'junior'
    return 'na'
    

df['title_category'] =  df['title'].apply(lambda x: jobtitle_cleaning(x) if isinstance(x, str) else 'na')
df['seniority'] =  df['title'].apply(lambda x: seniority_cleaning(x) if isinstance(x, str) else 'na')


In [75]:
df['title_category'].value_counts()
# df['seniority'].value_counts()


data scientist    1321
na                 108
analyst             41
data engineer        5
Name: title_category, dtype: int64

In [68]:
df.shape

(1475, 17)

In [73]:
df_w_salary = df[df['avg_salary'] != 0]
df_w_salary.shape
df_w_salary

,Unnamed: 0,title,company,location,company_rating,description,avg_salary,remote,state,python,aws,gcp,excel,spark,sql,title_category,seniority
14,14,Data Scientist- RAFP,Government of the District of Columbia,"Washington, DC 20005 (Downtown area)+1 location",3.6,A term appointment can last from 13 months and...,102359.5,0,DC,0,0,0,0,0,0,data scientist,na
18,18,Data Scientist I,Amazon.com Services LLC,+126 locationsRemote,3.5,"M.S. or equivalent in Statistics, Mathematics,...",126350.0,1,0,1,1,0,1,1,1,data scientist,na
26,26,Data Scientist,Cigna,"Remote in Hartford, CT",3.7,"Remote, Work from Home, United States\nPositio...",93900.0,1,CT,1,1,0,1,0,1,data scientist,na
29,29,"Data Scientist, Product (Remote)",Drizly,"Remote in Boston, MA",3.6,Who we are\nDrizly is the world’s largest alco...,110500.0,1,MA,1,0,0,0,0,1,data scientist,na
32,32,Data Scientist,State of Connecticut - Department of Public He...,"Hartford, CT",4.0,"The State of Connecticut, Department of Public...",108806.0,0,CT,1,0,0,1,0,0,data scientist,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,1425,Lead Data Scientist,S&P Global,"Cambridge, MA 02138 (West Cambridge area)+13 l...",3.9,NaN,136650.0,0,MA,0,0,0,0,0,0,data scientist,senior
1430,1430,"Research Scientist - HPC, WWPS Data and Machin...","Amazon Web Services, Inc.",+6 locationsRemote,3.5,NaN,125000.0,1,0,0,0,0,0,0,0,na,na
1434,1434,Data Scientist/Principal Data Scientist,Northrop Grumman,"Melbourne, FL+3 locations",4.0,NaN,97900.0,0,FL,0,0,0,0,0,0,data scientist,senior
1460,1460,Senior Data Scientist - Work from home-Evernorth,Cigna,"Hartford, CT 06152 (Frog Hollow area)+1 location",3.7,NaN,127700.0,0,CT,0,0,0,0,0,0,data scientist,senior
